In [1]:
!pip install pymysql
!pip install pymssql

import LibData
import LibUtils
import LibML
import LibFile
import LibEE

import pandas as pd
import numpy as np

jobName = '201216_Train_split1'

# 단계6. 임베딩2 :  원핫 -> 머니니스 -> 가입나이 -> 금리축소 -> 임베딩_상품코드 -> 임베딩_경과기간x현재나이 -> 스케일러
x, y = LibData.GetXYBase(jobName)
x = LibData.GetMoneyness(x)
x = LibData.GetAttainedAge(x)
x = LibData.GetIrReduction(x)
x = LibEE.GetX_Inter_EM_AA(x)
x = LibEE.GetXEmbApplied(3, x, y, ['ProdCode', 'Inter_EM_AA'])
scaler, x = LibUtils.ScaleStandard(x)

colsSelected = ['ElapsedMth', 'Sex', 'Prem', 'PremYr', 'AccumPrem', 'ContAmt', 'StartAge', 'FundAllo03', 'IR06', 'IR09', 'IR12', 'IR14', 'Moneyness', 'AttainedAge', 'ee_ProdCode_00', 'ee_ProdCode_01', 'ee_ProdCode_02']
x = x[colsSelected]

     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 1.3MB 5.4MB/s 


In [2]:
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import LibML

# 탐색 대상 함수 (XGBRegressor)
def XGB_cv(learning_rate, n_estimators, max_depth, subsample,min_child_weight, gamma):

    # 모델 정의
    model = XGBRegressor(learning_rate=learning_rate,
                         n_estimators=int(n_estimators),
                         max_depth=int(max_depth),
                         subsample=subsample,                         
                         min_child_weight=min_child_weight,
                         gamma = gamma,
                         nthread=2,
                         objective = 'reg:squarederror')
    
    # 모델 훈련 & 예측값
    scores = LibML.GetPred(x,y,model,2)
    
    # return negative mae
    return -scores[0] 

In [ ]:
#  bayesian-optimization 라이브러리의 BayesianOptimization 클래스 import
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization
import numpy as np

# 실험해보고자하는 hyperparameter 집합
# pbounds = { 'learning_rate': (0.01, 0.2),
#             'n_estimators': (5000, 10000),
#             'max_depth': (3, 10),
#             'subsample': (0, 1),
#             'colsample_bytree' : (0, 1),
#             'min_child_weight': (0, 4),
#             'max_delta_step' : (0, 1),
#             'gamma': (0, 100)
#             }

# 1차 
pbounds = {'gamma' : (0.0, 0.01),           
     'learning_rate': (0.0, 0.2),            
            'max_depth': (5, 15),
            'min_child_weight': (0, 4),
           'n_estimators': (3000, 5000),
           'subsample': (0.2, 0.7),
            }

# Bayesian optimization 객체 생성
# f : 탐색 대상 함수, pbounds : hyperparameter 집합
# verbose = 2 항상 출력, verbose = 1 최댓값일 때 출력, verbose = 0 출력 안함
# random_state : Bayesian Optimization 상의 랜덤성이 존재하는 부분을 통제 
bo=BayesianOptimization(f=XGB_cv, pbounds=pbounds, verbose=2, random_state=1)

# 메소드를 이용해 최대화 과정 수행
# init_points :  초기 Random Search 갯수
# n_iter : 반복 횟수 (몇개의 입력값-함숫값 점들을 확인할지! 많을 수록 정확한 값을 얻을 수 있다.)
# acq : Acquisition Function들 중 Expected Improvement(EI) 를 사용
# xi : exploration 강도 (기본값은 0.0)

bo.maximize(init_points=2, n_iter=70, acq='ei', xi=0.01)

# ‘iter’는 반복 회차, ‘target’은 목적 함수의 값, 나머지는 입력값을 나타냅니다. 
# 현재 회차 이전까지 조사된 함숫값들과 비교하여, 현재 회차에 최댓값이 얻어진 경우, 
# bayesian-optimization 라이브러리는 이를 자동으로 다른 색 글자로 표시하는 것을 확인할 수 있습니다

# 찾은 파라미터 값 확인
print(bo.max)

LibUtils.ELoop()

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=54ad586e7ca25abecb8b514282c21ff299e0bdfea20886d1dbf3e78fe376f3ea
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization
|   iter    |  target   |   gamma   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------
|  1        | -112.5    |  0.00417  |  0.1441   |  5.001    |  1.209    |  3.294e+0 |  0.2462   |
|  2        | -82.84    |  0.001863 |  0.06911  |  8.968    |  2.155    |  3.838e+0 |  0.5426   |
|  3        | -114.2    |  0.004007 |  0.1579   |  8.615    |  2.114    |  3.838e+0 |  0.2213   |
|  4        | -85.69    |  0.005977 |  0.06368  |  11.63    |  1.794    |  3.344e+0 |  0.3515   |
|  5        | -103.9    |  0.008972 |  0.1059   |  5.59     |  3.349    |  3.454e+0

KeyboardInterrupt: ignored